In [20]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
import time
import pydde as d

In [44]:
#Parameters
samplenum = 10
epochs = 100
hiddenlayers = [90]
input_size = 3
output_size = 3
learning_rate = 0.001
time_length = 60; #seconds

In [39]:
# Generate simulation
dyn = d.PyDyn('test2.sim', time_length)
state_init = dyn.compute(dyn.p_init)
f = dyn.f(state_init, dyn.p_init)
df = dyn.df_dp(state_init, dyn.p_init)
dy = dyn.dy_dp(state_init, dyn.p_init)

In [40]:
#Sample targets only variables in z direction
y_target = np.zeros((samplenum,3))
y_target[:,2] = np.random.rand(samplenum)
#x[:,0] = np.random.rand(samplenum)
y_target[:,1] = 2
y_target= torch.tensor(y_target)

## Building the custon Simulation Activation Function

In [41]:
class Simulate(torch.autograd.Function):
    
    @staticmethod
    def forward(ctx, input):
        #print(f'input: {input.shape}')
        p = input.clone().numpy().transpose()
        state = dyn.compute(p)
        y_pred = torch.tensor(state.y[-3:])
        #print(f'y_pred: {y_pred.dtype}')
        
        ctx.save_for_backward(input)
        
        return y_pred
    
    @staticmethod
    def backward(ctx, grad_output):
        #print(grad_output.shape)
        input, = ctx.saved_tensors
        p = input.clone().numpy().transpose()
        state= dyn.compute(p)
        dy_dp = dyn.dy_dp(state, p)
        dy_dp = dy_dp[-3:, :]
        #print(f'shape of dy/dp: {dy_dp.shape}')
        #print(f'shape of grad_output: {grad_output.shape}')
        grad_output = grad_output.unsqueeze(0)
        
        grad_input = torch.tensor(dy_dp).t().mm(grad_output.t()).t()
        return grad_input

Simulate = Simulate.apply

## Building the Model

In [42]:
class ActiveLearn(nn.Module):

    def __init__(self, n_in, out_sz):
        super(ActiveLearn, self).__init__()

        self.L_in = nn.Linear(n_in, hiddenlayers[0])
        self.H1 = nn.Linear(hiddenlayers[0], 3*time_length)
        #self.H1 = nn.Linear(hiddenlayers[0], hiddenlayers[1])
        #self.H2 = nn.Linear(hiddenlayers[1], 3*time_length)
        self.P = nn.Linear(3*time_length, 3*time_length)
        self.Relu = nn.ReLU(inplace=True)
    
    def forward(self, input):
        x = self.L_in(input)
        x = self.Relu(x)
        x = self.H1(x)
        x = self.Relu(x)
        #x = self.H2(x)
        #x = self.Relu(x)
        x = self.P(x)
        x = self.Relu(x)
        return x


model = ActiveLearn(input_size, output_size)

criterion = nn.MSELoss()  # RMSE = np.sqrt(MSE)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

y_target = y_target.float()

## Train the model

In [45]:
torch.autograd.set_detect_anomaly(True)

start_time = time.time()

losses = []
y_preds= np.zeros((samplenum, 3))
p_preds= np.zeros((samplenum, 3*time_length))

#y_pred = torch.tensor(y_pred)
for i in range(epochs):
    for s in range(samplenum):
        y_truth = y_target[s, :]
        #print(y_truth.shape)
        #y_truth = y_truth.unsqueeze(0)
        p_pred = model(y_truth)
        y_pred = Simulate(p_pred)
        #print(y_pred.shape)
        y_preds[s, :] = y_pred.detach()
        p_preds[s, :] = p_pred.detach()
        #loss = torch.sqrt(criterion(y_pred.float(), y_truth)) # RMSE
        loss = criterion(y_pred.float(), y_truth)  # MSE
        #loss =sum((y_pred.float()-y_truth)**2) 
        losses.append(loss)
        optimizer.zero_grad()
        #Back Prop
        loss.backward()
        optimizer.step()
    print(f'epoch: {i:3}/{epochs}  loss: {loss.item():10.8f}')
    i+=1

print(f'epoch: {i:3} loss: {loss.item():10.8f}') # print the last line
print(f'\nDuration: {(time.time() - start_time)/60:.3f} min') # print the time elapsed

epoch:   0/100  loss: 0.04818699
epoch:   1/100  loss: 0.01876822
epoch:   2/100  loss: 0.02525770
epoch:   3/100  loss: 0.01432685
epoch:   4/100  loss: 0.01568288
epoch:   5/100  loss: 0.01363829
epoch:   6/100  loss: 0.01301422
epoch:   7/100  loss: 0.01146603
epoch:   8/100  loss: 0.01235490
epoch:   9/100  loss: 0.01256030
epoch:  10/100  loss: 0.01293924
epoch:  11/100  loss: 0.01377768
epoch:  12/100  loss: 0.02032741
epoch:  13/100  loss: 0.01229425
epoch:  14/100  loss: 0.01037918
epoch:  15/100  loss: 0.01485858
epoch:  16/100  loss: 0.01882355
epoch:  17/100  loss: 0.01597444
epoch:  18/100  loss: 0.01202891
epoch:  19/100  loss: 0.01327722
epoch:  20/100  loss: 0.01909305
epoch:  21/100  loss: 0.01484646
epoch:  22/100  loss: 0.01606056
epoch:  23/100  loss: 0.01034472
epoch:  24/100  loss: 0.00953584
epoch:  25/100  loss: 0.00967595
epoch:  26/100  loss: 0.01608574
epoch:  27/100  loss: 0.02550515
epoch:  28/100  loss: 0.01566953
epoch:  29/100  loss: 0.02959391
epoch:  30

In [8]:
#Save Model

if len(losses) == epochs*(samplenum):
    torch.save(model.state_dict(), 'Trained_Model_300420_300s_100e_onlyZpos.pt')
    print('Model saved')
else:
    print('Model has not been trained. Consider loading a trained model instead.')

Model saved


## Test forward propagation

In [36]:
y_target= torch.tensor([0, 2, 0.5])
p = model(y_target)
y_pred = Simulate(p)
y_pred = y_pred.detach().numpy()
p = p.detach().numpy()


yTraj_test = dyn.compute(p)

print(y_pred)
print(y_target)
print(yTraj_test.y[-3:])
print(np.sum(yTraj_test.y[-3:]-y_pred))
print(p)

[0.0944146 1.9530724 0.3841143]
tensor([0.0000, 2.0000, 0.5000])
[0.0944146 1.9530724 0.3841143]
0.0
[0.         0.         0.         0.         0.         0.20432281
 0.         0.         1.451682   1.3087382  0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         1.6551723  0.         0.         0.
 0.         0.         0.07099061 0.         0.         0.
 0.         0.         0.         0.25406152 0.         0.
 0.         0.         0.         1.1448839  1.0023524  0.
 0.         0.         0.         0.         0.         0.
 0.         0.07773931 0.         0.         0.         0.1156168
 0.         0.         0.         0.         0.         0.
 0.         0.         0.5394917  0.         0.40568525 0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.01849897 0.         0.26445943 0.         0.
 0.         0.         0.         0.         0.         1.

## Torch Script Conversion and Saving

In [36]:
input_example = torch.tensor(y_target[0,:])
traced_script_module = torch.jit.trace(model, input_example)
original = model(test_input)

# Test the torch script
test_input= torch.tensor([0, 2, 0.5])
output_example = traced_script_module(test_input)
print(output_example[-12:])
print(original)

tensor([1.9574, 0.0000, 0.0000, 2.7701, 0.0000], grad_fn=<SliceBackward>)


In [ ]:
# Save serialized model
traced_script_module.save("CPP_example_model_latest.pt")